In [1]:
import numpy as np
from sklearn.decomposition import DictionaryLearning
import pickle

In [2]:
with open('/scratch/gpfs/yc6206/representation-engineering/examples/extraction/reading_vectors.pickle', 'rb') as f:
    reading_vectors = pickle.load(f)

In [3]:
dict_learner = DictionaryLearning(n_components=15, transform_algorithm='lasso_lars', transform_alpha=0.1, random_state=42)

In [4]:
X = reading_vectors['Happy'][-5]
X_transformed = dict_learner.fit(X).transform(X)

In [5]:
emotion_levels = {
    '-': 'slightly ',
    '': '',
    '+': 'very ',
    '++': 'extremely ',
    '+++': 'extremely extremely ',
    '++++': 'extremely extremely extremely ',
    '+++++': 'extremely extremely extremely extremely ',
    '++++++': 'extremely extremely extremely extremely extremely '
}
emotion_stimulis = {
    "Happy": "Respond with a happy tone",
    "Sad": "Respond with a sad tone",
    "Angry": "Respond with an angry tone",
    "Fearful": "Respond with a fearful tone",
    "Disgusted": "Respond with a disgusted tone",
    "Surprised": "Respond with a surprised tone"
}
emotions = list(emotion_stimulis.keys())

[[ 0.01237647 -0.00075446  0.02878658 ... -0.01693045 -0.01358844
   0.00923566]
 [ 0.01224699 -0.00071586  0.02846045 ... -0.01714701 -0.01356378
   0.0091946 ]
 [ 0.01213001 -0.000548    0.0286858  ... -0.01724414 -0.01335988
   0.00927199]
 ...
 [ 0.0121003  -0.00067088  0.02894696 ... -0.01703179 -0.0132918
   0.00921113]
 [ 0.01196598 -0.00075381  0.02867174 ... -0.01686708 -0.01345057
   0.00909359]
 [ 0.01185365 -0.00050974  0.0287874  ... -0.01723944 -0.01365963
   0.00935389]]


: 